In [1]:
import pandas as pd
from dhis2 import Api

from src.main.common import FileUtil
from src.main.common import LoggingConfig
from src.main.application.income import *
from src.main.application.service import *
from src.infrastructure.forms import *
from src.infrastructure.adapters import *
from src.main.application.out import *

In [2]:
credentials = FileUtil.load_credentias()
username = credentials['username']
password = credentials['password']

url = 'https://dhis-ao.icap.columbia.edu'
api = Api(url, username, password)

In [3]:
# Get the list of supported facilities
org_units = api.get('organisationUnitGroups/gH2DlwAo1ja', params={'fields':'organisationUnits'}).json()['organisationUnits']

In [4]:
# period of analysis
start_period = '2024-04-01'
end_period = '2024-06-30'
period = '2024Q2'

In [5]:
# Logger
config = FileUtil.load_logging_config()
logging_config = LoggingConfig(config)
logger = logging_config.logging_setup()

In [6]:
# index case
index_enrollments = pd.DataFrame(columns=['trackedEntity', 'program', 'status', 'orgUnit', 'patientIdentifier', 'patientAge', 'patientSex','patientName', 'childrenLessThan15Years', 'testPartner', 'numberOfContacts'])
index_enrollments.to_csv('INDEX_ENROLLMENTS.csv', index=False)

# Load patients envents
patient_demographics = PatientDemographicForm(api)
patient_events_port = PatientEventForm(api)

for org_unit in org_units:

    org_unit = org_unit['id']

    #get all patient events
    patients_events = patient_events_port.find_patient_events_by_program_program_stage_unit_and_period(INDEX, INDEX_DETAILS_STAGE, org_unit, start_period, end_period)

    logger.info(f"Processing the facility: {org_unit}, a total of {len(patients_events)} enrolled patient(s)")
    
    counter = 1

    for patient_enrolled in patients_events:
        patient_id = patient_enrolled['trackedEntity']

        patient_demographics.add_demographics(patient_enrolled)

        logger.info(f"From {len (patients_events)} patietnts enrolled, {counter} (is) are ready to be processed.")
        counter = counter + 1 

    if patients_events:
        patients_events = pd.json_normalize(patients_events)
        
        enrollments = pd.read_csv('INDEX_ENROLLMENTS.csv')
        enrollments = pd.concat([enrollments, patients_events])
        enrollments.to_csv('INDEX_ENROLLMENTS.csv', index=False, encoding='utf-8')

In [11]:
# index case contacts
index_enrollments = pd.DataFrame(columns=['trackedEntity', 'program', 'status', 'orgUnit', 'patientAge', 'patientSex', 'ic_new_case', 'result', 'ic_date'])
index_enrollments.to_csv('CONTACTS_INDEX_ENROLLMENTS.csv', index=False)

# Load patients envents
patient_demographics = PatientDemographicForm(api)
patient_events_port = PatientEventForm(api)

for org_unit in org_units:

    org_unit = org_unit['id']

    #get all patient events
    patients_events = patient_events_port.find_patient_events_by_program_program_stage_unit_and_period(INDEX, INDEX_CONTACTS_STAGE, org_unit, start_period, end_period)

    logger.info(f"Processing the facility: {org_unit}, a total of {len(patients_events)} enrolled patient(s)")

    if patients_events:
        patients_events = pd.json_normalize(patients_events)
        
        enrollments = pd.read_csv('CONTACTS_INDEX_ENROLLMENTS.csv')
        enrollments = pd.concat([enrollments, patients_events])
        enrollments.to_csv('CONTACTS_INDEX_ENROLLMENTS.csv', index=False, encoding='utf-8')

In [6]:
index_enrollments = pd.read_csv('INDEX_ENROLLMENTS.csv')
index_enrollments = index_enrollments.to_dict(orient='records')

# index
index_case_offered_indicators_metadata_port = IndexCaseOfferedIndicatorMetadataAdapter(api)
index_case_offered_use_case: ComputeHtsIndexCaseOfferedDisaggregationUseCase = ComputeHtsIndexCaseOfferedDisaggregationService(logger, index_case_offered_indicators_metadata_port)
index_case_offered_patients_disaggregations =  index_case_offered_use_case.compute(index_enrollments, end_period)

index_case_accepted_use_case = ComputeHtsIndexCaseAcceptedService()
index_case_accepted_patients = index_case_accepted_use_case.compute(index_enrollments)

index_case_accepted_indicators_metadata_port : IndicatorMetadataPort = IndexCaseAcceptedIndicatorMetadataAdapter(api)
index_case_accepted_disaggregation_use_case: ComputeHtsIndexCaseAcceptedDisaggregationUseCase = ComputeHtsIndexCaseAcceptedDisaggregationService(logger, index_case_accepted_indicators_metadata_port)
index_case_accepted_patients_disaggregation = index_case_accepted_disaggregation_use_case.compute(index_case_accepted_patients, end_period)

# contacts
index_contacts_enrollments = pd.read_csv('CONTACTS_INDEX_ENROLLMENTS.csv')
index_contacts_enrollments = index_contacts_enrollments.to_dict(orient='records')

index_case_contacts_number_indicators_metadata_port: IndicatorMetadataPort = IndexCaseContactsNumberIndicatorMetadataAdapter(api)
index_case_contacts_number_disaggregation_use_case: ComputeHtsIndexCaseContactsNumberDisaggregationUseCase = ComputeHtsIndexCaseContactsNumberDisaggregationService(logger, index_case_contacts_number_indicators_metadata_port)
index_case_contacts_number_patients_disaggregation = index_case_contacts_number_disaggregation_use_case.compute(index_contacts_enrollments, end_period)

index_case_use_case: ComputeHtsIndexCaseUseCase = ComputeHtsIndexCaseService()
index_case_patients = index_case_use_case.compute(index_contacts_enrollments)

index_case_indicators_metadata_port :IndicatorMetadataPort = IndexCaseIndicatorMetadataAdapter(api)
index_case_disaggregation_use_case: ComputeHtsIndexCaseDisaggregationUseCase = ComputeHtsIndexCaseDisaggregationService(logger,index_case_indicators_metadata_port)
index_case_patients_disaggregations = index_case_disaggregation_use_case.compute(index_case_patients, end_period)

combination = index_case_offered_patients_disaggregations + index_case_accepted_patients_disaggregation + index_case_contacts_number_patients_disaggregation + index_case_patients_disaggregations

if combination:
    # extract data
    indicators = pd.json_normalize(combination)
    indicators['period'] = period
    indicators = indicators[['dataElement','period','orgUnit', 'categoryOptionCombo','attributeOptionCombo', 'value']]
    indicators = indicators.sort_values(['orgUnit', 'dataElement', 'period'])
    indicators.to_csv('INDEX_DATA.csv', index=False)